In [1]:
import pandas as pd
import numpy as np
import os
import random
from itertools import permutations
from copy import copy

In [2]:
def generate_ranks(number):

    electability = list(range(1, number+1))
#     random.shuffle(electability)
    for i in range(len(electability)):
        electability[i] = 1/(electability[i]**2)
        
    print('electability', electability, [x+1 for x in np.argsort(electability)])
    
    general_fav = [random.random() for i in electability]

    proportion = 0.5
    truthful = []
    deviation = []
    for i in range(10000):
        favorability = [random.random()*general_fav[i] for i in range(len(electability))]
        truthful.append([x + 1 for x in np.argsort(favorability)][::-1])
        if random.random() < proportion:
            rank = [favorability[i]*electability[i] for i in range(number)]
            deviation.append([x + 1 for x in np.argsort(rank)][::-1])
        else:
            deviation.append([x + 1 for x in np.argsort(favorability)][::-1])
    return [truthful, deviation]

In [3]:
class Borda:

    def __init__(self, candidates, vote_count, preferences, pref_type):
        self.candidates = candidates
        self.vote_count = vote_count
        self.preferences = preferences
        self.pref_type = pref_type

    def determine_winner(self):
        vote_tally = [0] * len(self.candidates)
        preferences = self.preferences.copy()
        for p in range(len(preferences)):
            for i in range(len(preferences[p])):
                try:
                    vote_tally[preferences[p][i]-1] += float(1/(1+i))
                except:
                    print(preferences[p][i])

        idx = vote_tally.index(max(vote_tally))
        return self.candidates[idx+1]

In [4]:
class RankedChoice:

    def __init__(self, candidates, vote_count, preferences, pref_type):
        self.candidates = candidates
        self.vote_count = vote_count
        self.preferences = preferences
        self.pref_type = pref_type

    def determine_winner(self):
        vote_tally = [0] * len(self.candidates)
        candidates_left = list(self.candidates.keys())
        prefs = []
        for i in self.preferences:
            temp = []
            for j in i:
                temp.append(j)
            prefs.append(temp)
        while True:
            for p in range(len(prefs)):
                if len(prefs[p]) == 0:
                    continue
                try:
                    while prefs[p][0] not in candidates_left:
                        prefs[p].pop(0)
                        if len(prefs[p]) == 0:
                            continue
                except:
                    continue
                idx = candidates_left.index(prefs[p][0])
                vote_tally[idx]+=1
                
            if max(vote_tally) == min(vote_tally):
                return "Tie"
            elif max(vote_tally) > sum(vote_tally)/2:
                idx = vote_tally.index(max(vote_tally))
                winner_idx = candidates_left[idx]
                return self.candidates[winner_idx]
            else:
                idx = vote_tally.index(min(vote_tally))
                candidates_left.pop(idx)
                vote_tally = [0] * len(candidates_left)


In [5]:
class Plurality:

    def __init__(self, candidates, vote_count, preferences, pref_type):
        self.candidates = candidates
        self.vote_count = vote_count
        self.preferences = preferences
        self.pref_type = pref_type

    def determine_winner(self):
        vote_tally = [0] * len(self.candidates)
        preferences = self.preferences.copy()
        for p in range(len(preferences)):
            try:
                vote_tally[preferences[p][0]-1] += 1
            except:
                continue

        idx = vote_tally.index(max(vote_tally))
        return self.candidates[idx+1]

In [6]:
class TwoRound:

    def __init__(self, candidates, vote_count, preferences, pref_type):
        self.candidates = candidates
        self.vote_count = vote_count
        self.preferences = preferences
        self.pref_type = pref_type

    def determine_winner(self):
        vote_tally = [0] * len(self.candidates)
        preferences = self.preferences.copy()
        for p in range(len(preferences)):
            try:
                vote_tally[preferences[p][0]-1] += 1
            except:
                continue

        if max(vote_tally) > sum(vote_tally):
            idx = vote_tally.index(max(vote_tally))
            return self.candidates[idx]

        rank_order = list(np.argsort(vote_tally))[-2:]
        final_tally = [0,0]

        for p in range(len(preferences)):
            for i in range(len(preferences[p])):
                if preferences[p][i] in rank_order:
                    idx = rank_order.index(preferences[p][i])
                    final_tally[idx] += 1
                    continue
        if final_tally[0] > final_tally[1]:
            return self.candidates[rank_order[0]+1]
        elif final_tally[0] < final_tally[1]:
            return self.candidates[rank_order[1]+1]
        return "Tie"

In [7]:
candidate3 = {1: 'SpongeBob SquarePants',
 2: 'Patrick Star',
 3: 'Squidward Tentacles'}

In [8]:
candidate5 = {1: 'SpongeBob SquarePants',
 2: 'Patrick Star',
 3: 'Squidward Tentacles',
 4: 'Mr. Krabs',
 5: 'Plankton'}

In [9]:
candidate10 = {1: 'SpongeBob SquarePants',
 2: 'Patrick Star',
 3: 'Squidward Tentacles',
 4: 'Mr. Krabs',
 5: 'Plankton',
 6: 'Sandy Cheeks',
 7: 'Gary',
 8: 'Larry the Lobster',
 9: 'Pearl',
 10: 'Mrs. Puff'}

In [10]:
def calculate_stats(winner, truthful):
    total = 0
    count = 0
    last_count = 0
    first_count = 0
    for i in truthful:
        if winner in i:
            count += 1
            rank = i.index(winner) + 1
            total += rank
            if rank == len(i):
                last_count += 1
            if rank == 1:
                first_count += 1
    avg = float(total)/count
    last_prop = float(last_count)/len(truthful)
    first_prop = float(first_count)/len(truthful)
    return [avg, last_prop, first_prop]

In [11]:
truthful3, deviation3 = generate_ranks(3)

electability [1.0, 0.25, 0.1111111111111111] [3, 2, 1]


In [12]:
truthful5, deviation5 = generate_ranks(5)

electability [1.0, 0.25, 0.1111111111111111, 0.0625, 0.04] [5, 4, 3, 2, 1]


In [13]:
truthful10, deviation10 = generate_ranks(10)

electability [1.0, 0.25, 0.1111111111111111, 0.0625, 0.04, 0.027777777777777776, 0.02040816326530612, 0.015625, 0.012345679012345678, 0.01] [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


In [14]:
truthful10[:10]

[[10, 6, 9, 7, 3, 2, 4, 1, 8, 5],
 [10, 9, 4, 2, 6, 5, 3, 1, 7, 8],
 [4, 3, 9, 5, 6, 10, 2, 1, 8, 7],
 [10, 2, 5, 3, 9, 4, 1, 7, 6, 8],
 [10, 4, 6, 2, 5, 7, 9, 1, 3, 8],
 [9, 2, 5, 7, 10, 4, 1, 3, 6, 8],
 [5, 10, 4, 1, 6, 7, 9, 2, 8, 3],
 [10, 6, 5, 2, 7, 1, 3, 4, 9, 8],
 [6, 4, 5, 7, 9, 2, 3, 10, 1, 8],
 [9, 6, 2, 5, 10, 4, 3, 1, 7, 8]]

In [15]:
deviation10[:10]

[[10, 6, 9, 7, 3, 2, 4, 1, 8, 5],
 [1, 2, 3, 4, 5, 6, 9, 10, 7, 8],
 [1, 3, 4, 2, 5, 6, 9, 10, 8, 7],
 [1, 2, 3, 5, 4, 10, 9, 7, 6, 8],
 [10, 4, 6, 2, 5, 7, 9, 1, 3, 8],
 [9, 2, 5, 7, 10, 4, 1, 3, 6, 8],
 [5, 10, 4, 1, 6, 7, 9, 2, 8, 3],
 [10, 6, 5, 2, 7, 1, 3, 4, 9, 8],
 [6, 4, 5, 7, 9, 2, 3, 10, 1, 8],
 [1, 2, 3, 5, 6, 4, 9, 10, 7, 8]]

In [16]:
vote_count = 10000

In [17]:
true_choice_3B = Borda(candidate3, vote_count, truthful3, "soi").determine_winner()
deviation_choice_3B = Borda(candidate3, vote_count, deviation3, "soi").determine_winner()
true_choice_5B = Borda(candidate5, vote_count, truthful5, "soi").determine_winner()
deviation_choice_5B = Borda(candidate5, vote_count, deviation5, "soi").determine_winner()
true_choice_10B = Borda(candidate10, vote_count, truthful10, "soi").determine_winner()
deviation_choice_10B = Borda(candidate10, vote_count, deviation10, "soi").determine_winner()

In [18]:
true_choice_3P = Plurality(candidate3, vote_count, truthful3, "soi").determine_winner()
deviation_choice_3P = Plurality(candidate3, vote_count, deviation3, "soi").determine_winner()
true_choice_5P = Plurality(candidate5, vote_count, truthful5, "soi").determine_winner()
deviation_choice_5P = Plurality(candidate5, vote_count, deviation5, "soi").determine_winner()
true_choice_10P = Plurality(candidate10, vote_count, truthful10, "soi").determine_winner()
deviation_choice_10P = Plurality(candidate10, vote_count, deviation10, "soi").determine_winner()

In [19]:
true_choice_3R = RankedChoice(candidate3, vote_count, truthful3, "soi").determine_winner()
deviation_choice_3R = RankedChoice(candidate3, vote_count, deviation3, "soi").determine_winner()
true_choice_5R = RankedChoice(candidate5, vote_count, truthful5, "soi").determine_winner()
deviation_choice_5R = RankedChoice(candidate5, vote_count, deviation5, "soi").determine_winner()
true_choice_10R = RankedChoice(candidate10, vote_count, truthful10, "soi").determine_winner()
deviation_choice_10R = RankedChoice(candidate10, vote_count, deviation10, "soi").determine_winner()

In [20]:
true_choice_3T = TwoRound(candidate3, vote_count, truthful3, "soi").determine_winner()
deviation_choice_3T = TwoRound(candidate3, vote_count, deviation3, "soi").determine_winner()
true_choice_5T = TwoRound(candidate5, vote_count, truthful5, "soi").determine_winner()
deviation_choice_5T = TwoRound(candidate5, vote_count, deviation5, "soi").determine_winner()
true_choice_10T = TwoRound(candidate10, vote_count, truthful10, "soi").determine_winner()
deviation_choice_10T = TwoRound(candidate10, vote_count, deviation10, "soi").determine_winner()

In [21]:
true_choice_3B, deviation_choice_3B, true_choice_5B, deviation_choice_5B, true_choice_10B, deviation_choice_10B,

('Patrick Star',
 'SpongeBob SquarePants',
 'Patrick Star',
 'SpongeBob SquarePants',
 'Mrs. Puff',
 'SpongeBob SquarePants')

In [22]:
for i in range(1,4):
    print(candidate3[i])
    print(calculate_stats(i, truthful3))

SpongeBob SquarePants
[1.7836, 0.0768, 0.2932]
Patrick Star
[1.3381, 0.0397, 0.7016]
Squidward Tentacles
[2.8783, 0.8835, 0.0052]


In [23]:
for i in range(1,6):
    print(candidate5[i])
    print(calculate_stats(i, truthful5))

SpongeBob SquarePants
[2.2425, 0.0761, 0.3634]
Patrick Star
[2.0571, 0.0725, 0.4749]
Squidward Tentacles
[2.9014, 0.1215, 0.1207]
Mr. Krabs
[3.3767, 0.166, 0.0399]
Plankton
[4.4223, 0.5639, 0.0011]


In [24]:
for i in range(1,11):
    print(candidate10[i])
    print(calculate_stats(i, truthful10))

SpongeBob SquarePants
[6.5036, 0.0542, 0.002]
Patrick Star
[4.2764, 0.0268, 0.1766]
Squidward Tentacles
[6.8634, 0.0668, 0.0013]
Mr. Krabs
[5.3712, 0.0396, 0.0286]
Plankton
[4.5896, 0.0297, 0.1124]
Sandy Cheeks
[4.5231, 0.0311, 0.129]
Gary
[5.3371, 0.039, 0.0339]
Larry the Lobster
[9.5707, 0.662, 0.0]
Pearl
[4.2701, 0.0267, 0.1798]
Mrs. Puff
[3.6948, 0.0241, 0.3364]
